# ETL

## Extract

In [252]:
import pandas as pd

# Create dataframes
subject_df = pd.read_csv('../resources/subject.txt', sep=',')
colors_df = pd.read_csv('../resources/colors.txt', sep=',')
dates_df = pd.read_csv('../resources/dates.txt', sep=',', names=['title', 'date', 'other'])

## Transform

In [253]:
# Delete unnecessary columns
subject_df = subject_df.drop(['EPISODE', 'TITLE', 'GUEST', 'DIANE_ANDRE', 'STEVE_ROSS'], axis=1)
subject_df.drop(list(subject_df.filter(regex = 'FRAME')), axis = 1, inplace = True)

# Create list of all subjects (in columns 2 onwards)
cols = subject_df.columns.tolist()[1:]
# If any subject column doesn't include any paintings, delete it
for col in cols:
	if (subject_df[col].mean() == 0):
		subject_df = subject_df.drop(col, axis=1)

# Create same index in all dataframes
subject_df['id'] = range(0, len(subject_df))

subject_df.head(2)

,AURORA_BOREALIS,BARN,BEACH,BOAT,BRIDGE,BUILDING,BUSHES,CABIN,CACTUS,CIRRUS,...,SNOWY_MOUNTAIN,STRUCTURE,SUN,TREE,TREES,WATERFALL,WAVES,WINDMILL,WINTER,id
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,1,0,0,0,1,1


In [254]:
# Delete unnecessary columns
colors_df.drop(['Unnamed: 0', 'painting_index', 'season', 'episode', 'color_hex', 'colors', 'painting_title'], axis=1, inplace=True)

# Find list of all possible colors
cols = colors_df.columns.tolist()[3:]
# Sum number of colors for each painting
colors_df['verify_colors'] = colors_df[cols].sum(axis=1)
# Verify that calculated number of colors matches given number of colors
colors_df.loc[~(colors_df['verify_colors'] == colors_df['num_colors'])]

# Since verification passed, delete both columns
colors_df.drop(['verify_colors', 'num_colors'], axis=1, inplace=True)

# Create same index in all dataframes
colors_df['id'] = range(0, len(colors_df))

colors_df.head(2)

,img_src,youtube_src,Black_Gesso,Bright_Red,Burnt_Umber,Cadmium_Yellow,Dark_Sienna,Indian_Red,Indian_Yellow,Liquid_Black,...,Midnight_Black,Phthalo_Blue,Phthalo_Green,Prussian_Blue,Sap_Green,Titanium_White,Van_Dyke_Brown,Yellow_Ochre,Alizarin_Crimson,id
0,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/oh5p5f5_-7A,0,1,0,1,0,0,0,0,...,0,0,1,1,1,1,1,0,1,0
1,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/RInDWhYceLU,0,1,0,1,0,0,0,0,...,0,0,1,1,1,1,1,0,1,1


In [255]:
dates_df.drop(['other'], axis=1, inplace=True)
dates_df['id'] = range(0, len(dates_df))

# id should be first column
dates_df = dates_df[['id', 'title', 'date']]

dates_df.head(2)

,id,title,date
0,0,A Walk in the Woods,January 11 1983
1,1,Mount McKinley,January 11 1983


In [256]:
# Merge dataframes based on ep_id
two_df = pd.merge(dates_df, colors_df)
three_df = pd.merge(two_df, subject_df)

# All column names are lowercse
three_df.columns = three_df.columns.str.lower()

three_df.head(2)

,id,title,date,img_src,youtube_src,black_gesso,bright_red,burnt_umber,cadmium_yellow,dark_sienna,...,snow,snowy_mountain,structure,sun,tree,trees,waterfall,waves,windmill,winter
0,0,A Walk in the Woods,January 11 1983,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/oh5p5f5_-7A,0,1,0,1,0,...,0,0,0,0,1,1,0,0,0,0
1,1,Mount McKinley,January 11 1983,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/RInDWhYceLU,0,1,0,1,0,...,1,1,0,0,1,1,0,0,0,1


## Load

In [257]:
# # Since db/table created in engine/db.py, don't need to create it here!
# # So just for reference on how to possibly create within JNB

# text_cols = three_df.columns.tolist()[1:5]
# int_cols = three_df.columns.tolist()[5:]

# col_list = ['id INTEGER NOT NULL PRIMARY KEY']

# # Add text columns to list
# for col in text_cols:
# 	col_list.append(col + ' TEXT')

# # Add integer columns to list
# for col in int_cols:
# 	col_list.append(col + ' INTEGER')

# col_str = ', '.join(col_list)

# # If database exists, connect to it - otherwise create it and then connect
# conn = sqlite3.connect('../bob_ross.db')
# cursor = conn.cursor()
# # Create table with column names and values based on col_str
# cursor.execute('CREATE TABLE IF NOT EXISTS paintings(col_str)')

In [258]:
import sqlite3
import sqlalchemy

# Connect to database
engine = sqlalchemy.create_engine('sqlite:///../bob_ross.db')

# Add data in dataframe to table
three_df.to_sql('episodes', engine, if_exists='replace', index=False)

## DB Queries

In [259]:
# pd.read_sql('episodes', engine)
# pd.read_sql('SELECT img_src FROM episodes where apple_frame=1', engine)
# pd.read_sql('SELECT title, date, img_src FROM paintings WHERE date LIKE "%January%"', engine)